Datasets are from [Hejing Community](https://www.kesci.com/mw/dataset/5ffac64f3441fd001538228b/file), we devide the data into three parts.

# 1.Order dataset

## 1.1 Data cleaning

In [1]:
import pandas as pd

In [2]:
# read csv file
data = pd.read_csv('/Users/will/Documents/LandislandGithub/DataAnalysis/TMallOrder/tmall_order_report.csv')

In [7]:
# data at first glance
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


In [8]:
# this dataset contains
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


We can tell from above in this dataset contains 28010 entries and 7 columns.

In [11]:
data.columns = data.columns.str.strip()  # deal with space in column name
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

In [12]:
# check if there is duplicate data
data[data.duplicated()].count()

订单编号        0
总金额         0
买家实际支付金额    0
收货地址        0
订单创建时间      0
订单付款时间      0
退款金额        0
dtype: int64

There is no duplicate data

In [13]:
data.isnull().sum()   # 付款时间存在空值，表示订单未付款

订单编号           0
总金额            0
买家实际支付金额       0
收货地址           0
订单创建时间         0
订单付款时间      3923
退款金额           0
dtype: int64

There are 3923 records whose `order pay time` is null, which means customer did not pay.

In [17]:
data['收货地址'] = data['收货地址'].str.replace('自治区|维吾尔|回族|壮族|省', '')  # just pure province name, without xx `province`
data['收货地址'].unique()

<ipython-input-17-06448c9ee336>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['收货地址'] = data['收货地址'].str.replace('自治区|维吾尔|回族|壮族|省', '')  # just pure province name, without xx `province`


array(['上海', '内蒙古', '安徽', '湖南', '江苏', '浙江', '天津', '北京', '四川', '贵州', '辽宁',
       '河南', '广西', '广东', '福建', '海南', '江西', '甘肃', '河北', '黑龙江', '云南', '重庆',
       '山西', '吉林', '山东', '陕西', '湖北', '青海', '新疆', '宁夏', '西藏'], dtype=object)

## 1.2 Data visualization

### 1.2.1 Data overview

In [22]:
result = {}
result['Number of total orders'] = data['订单编号'].count()  
result['Number of finish orders'] = data['订单编号'][data['订单付款时间'].notnull()].count()  
result['Number of unpaid orders'] = data['订单编号'][data['订单付款时间'].isnull()].count()  
result['Number of refund orders'] = data['订单编号'][data['退款金额'] > 0].count()  
result['Number of total sales'] = data['总金额'][data['订单付款时间'].notnull()].sum()  
result['Number of total refund'] = data['退款金额'][data['订单付款时间'].notnull()].sum()  
result['Number of revenue'] = data['买家实际支付金额'][data['订单付款时间'].notnull()].sum()  

In [23]:
result

{'Number of total orders': 28010,
 'Number of finish orders': 24087,
 'Number of unpaid orders': 3923,
 'Number of refund orders': 5646,
 'Number of total sales': 2474823.0700000003,
 'Number of total refund': 572335.9199999999,
 'Number of revenue': 1902487.1500000001}

In [28]:
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker

table = Table()

headers = ['# of total orders', '# of total sales', '# of finish orders', '# of revenue', 'Number of refund orders', '# of total refund', 'Coversion rate', 'Return rate']
rows = [
    [
        result['Number of total orders'], f"{result['Number of total sales']/1000:.2f} K", result['Number of finish orders'], f"{result['Number of revenue']/1000:.2f} K",
        result['Number of refund orders'], f"{result['Number of total refund']/1000:.2f} K", 
        f"{result['Number of finish orders']/result['Number of total orders']:.2%}",
        f"{result['Number of refund orders']/result['Number of finish orders']:.2%}",
    ]
]
table.add(headers, rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='Overview')
)
table.render_notebook()

# of total orders,# of total sales,# of finish orders,# of revenue,Number of refund orders,# of total refund,Coversion rate,Return rate
28010,2474.82 K,24087,1902.49 K,5646,572.34 K,85.99%,23.44%


### 1.2.2 Area distribution

In [31]:
result2 = data[data['订单付款时间'].notnull()].groupby('收货地址').agg({'订单编号':'count'})
result21 = result2.to_dict()['订单编号']
c = (
    Map()
    .add("订单量", [*result21.items()], "china", is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='Area distribution'),
        visualmap_opts=opts.VisualMapOpts(max_=1000),            
    )
)
c.render_notebook()

### 1.2.3 Time series analysis

In [34]:
data['订单创建时间'] = pd.to_datetime(data['订单创建时间'])
data['订单付款时间'] = pd.to_datetime(data['订单付款时间'])

In [35]:
result31 = data.groupby(data['订单创建时间'].apply(lambda x: x.strftime("%Y-%m-%d"))).agg({'订单编号':'count'}).to_dict()['订单编号']
c = (
    Line()
    .add_xaxis(list(result31.keys()))
    .add_yaxis("Number of orders", list(result31.values()))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="Maximum"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Number of orders per day over time"))
)
c.render_notebook()

We can tell from the pic above, during the begining of COVID-19(2020-01 to 2020-02), the orders shrunk sharply.

In [37]:
result32 = data.groupby(data['订单创建时间'].apply(lambda x: x.strftime("%H"))).agg({'订单编号':'count'}).to_dict()['订单编号']
x = [*result32.keys()]
y = [*result32.values()]
c = (
    Bar()
    .add_xaxis(x)
    .add_yaxis("Number of orders", y)
    .set_global_opts(title_opts=opts.TitleOpts(title="Number of orders per hour over time"))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="apex"),
                opts.MarkPointItem(name="second apex", coord=[x[15], y[15]], value=y[15]),
                opts.MarkPointItem(name="third apex", coord=[x[10], y[10]], value=y[10]),
            ]
        ),
    )
)
c.render_notebook()

The top 3 busy hour are 10:00AM, 15:00PM and 21:00PM. This can tell us to put more online service staff during the night.

# 2. Double 11 Beauty sale dataset

## 2.1 Data interpretation

In [40]:
data2 = pd.read_csv('/Users/will/Documents/LandislandGithub/DataAnalysis/TMallOrder/double11beautysales.csv')
data2.head()

,update_time,id,title,price,sale_count,comment_count,店名
0,2016/11/14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2016/11/14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2016/11/14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2016/11/14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2016/11/14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂


In [41]:
data2.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598 entries, 0 to 27597
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   update_time    27598 non-null  object 
 1   id             27598 non-null  object 
 2   title          27598 non-null  object 
 3   price          27598 non-null  float64
 4   sale_count     25244 non-null  float64
 5   comment_count  25244 non-null  float64
 6   店名             27598 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.5+ MB


This dataset contains 27598 entries and 7 columns.

In [43]:
data2[data2.duplicated()].count() # calculate duplicate data

update_time      86
id               86
title            86
price            86
sale_count       82
comment_count    82
店名               86
dtype: int64

In [44]:
data2.drop_duplicates(inplace=True)   # delete duplicate data
data2.reset_index(drop=True, inplace=True)  # rebuild index

In [45]:
data2.isnull().sum() 

update_time         0
id                  0
title               0
price               0
sale_count       2350
comment_count    2350
店名                  0
dtype: int64

In [46]:
data2.fillna(0, inplace=True) # fill null value

#formatlize time
data2['update_time'] = pd.to_datetime(data2['update_time']).apply(lambda x: x.strftime("%Y-%m-%d")) 

In [47]:
data2['sale_amount'] = data2['price'] * data2['sale_count']  # add a new column
data2[data2['sale_count']>0].sort_values(by=['sale_count'])

,update_time,id,title,price,sale_count,comment_count,店名,sale_amount
27042,2016-11-05,A541190557158,Herborist/佰草集新美肌梦幻曲面贴膜3片 保湿补水,1.0,1.0,0.0,佰草集,1.0
1494,2016-11-10,A538981087285,【双II预售】资生堂 新透白色控霜 30ml,390.0,1.0,0.0,资生堂,390.0
24148,2016-11-09,A540190519057,【娇兰盛典】腮红亲密容和肌肤 裸妆感 自然持久玫瑰闰色腮红,420.0,1.0,0.0,娇兰,420.0
24147,2016-11-09,A540189922026,【娇兰盛典】丝柔蜜粉饼 营造细致透明妆感 柔滑细腻贴肤美颜,480.0,1.0,1.0,娇兰,480.0
16974,2016-11-05,A541166044768,L'OREAL欧莱雅卓韵霜时尚魅棕系列染发霜 富含炫闪因子蜜茶棕红棕,79.0,1.0,0.0,欧莱雅,79.0
...,...,...,...,...,...,...,...,...
17470,2016-11-10,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1827562.0,200154.0,妮维雅,76757604.0
17339,2016-11-11,A24304992630,2瓶更划算*妮维雅男士洗面奶控油祛痘印保湿去黑头去油洁面乳护肤,35.0,1886100.0,199532.0,妮维雅,66013500.0
17228,2016-11-12,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,37.9,1920083.0,199062.0,妮维雅,72771145.7
17126,2016-11-13,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,37.9,1921582.0,198774.0,妮维雅,72827957.8


## 2.2 Data visulization

### 2.2.1 Number of sales per day over time

In [50]:
result = data2.groupby('update_time').agg({'sale_count':'sum'}).to_dict()['sale_count']
c = (
    Line()
    .add_xaxis(list(result.keys()))
    .add_yaxis("Number of sales", list(result.values()))
    .set_series_opts(
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="Max value"),
                opts.MarkPointItem(type_="min", name="Min value"),
                opts.MarkPointItem(type_="average", name="Average value"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title=" Number of sales per day over time"))
)
c.render_notebook()

### 2.2.2 Best seller

In [52]:
dts = list(data2['update_time'].unique())
dts.reverse()
dts

['2016-11-05',
 '2016-11-06',
 '2016-11-07',
 '2016-11-08',
 '2016-11-09',
 '2016-11-10',
 '2016-11-11',
 '2016-11-12',
 '2016-11-13',
 '2016-11-14']

In [55]:
from pyecharts import options as opts
from pyecharts.charts import Map, Timeline, Bar, Line, Pie
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts

tl = Timeline()
tl.add_schema(
#         is_auto_play=True,
        is_loop_play=False,
        play_interval=500,
    )
for dt in dts:
    item = data2[data2['update_time'] <= dt].groupby('店名').agg({'sale_count': 'sum', 'sale_amount': 'sum'}).sort_values(by='sale_count', ascending=False)[:10].sort_values(by='sale_count').to_dict()
    bar = (
        Bar()
        .add_xaxis([*item['sale_count'].keys()])
        .add_yaxis("Number of sales", [round(val/1000,2) for val in item['sale_count'].values()], label_opts=opts.LabelOpts(position="right", formatter='{@[1]/} K'))
        .add_yaxis("Sales", [round(val/10000/100,2) for val in item['sale_amount'].values()], label_opts=opts.LabelOpts(position="right", formatter='{@[1]/} Million'))
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts("Cumulative Sales TOP10")
        )
    )
    tl.add(bar, dt)
tl.render_notebook()

In [56]:
item = data2.groupby('店名').agg({'sale_count': 'sum'}).sort_values(by='sale_count', ascending=False)[:10].to_dict()['sale_count']
item = {k: round(v/10000, 2) for k, v in item.items()}
c = (
    Pie()
    .add("Number of Sales", [*item.items()])
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c} K({d}%)"))
)
c.render_notebook()

### 2.2.3 Price Ranking

In [57]:
item = data2.groupby('店名').agg({'price': 'mean'}).sort_values(by='price', ascending=False)[:20].sort_values(by='price').to_dict()
c = (
    Bar()
    .add_xaxis([*item['price'].keys()])
    .add_yaxis("Number of sales", [round(v, 2) for v in item['price'].values()], label_opts=opts.LabelOpts(position="right"))
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts("Average Price TOP20")
    )
)
c.render_notebook()

# 3. Fragrance Dataset

In this "fragrance.xlsx" file, it contains 2 sheet: one is order sheet and the other is product sheet.

## 3.1 Data Interpretation

In [60]:
import pandas as pd 
fact_order = pd.read_excel('/Users/will/Documents/LandislandGithub/DataAnalysis/TMallOrder/fragrance.xlsx', sheet_name='销售订单表')
dim_product = pd.read_excel('/Users/will/Documents/LandislandGithub/DataAnalysis/TMallOrder/fragrance.xlsx', sheet_name='商品信息表')

In [68]:
# for fact_order sheet
fact_order.head()

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额
0,D31313,2019-05-16 00:00:00,S22796,东区,浙江省,台州市,X091,892,214,190888.0
1,D21329,2019-05-14 00:00:00,S11460,东区,安徽省,宿州市,X005,276,185,51060.0
2,D22372,2019-08-26 00:00:00,S11101,北区,山西省,忻州市,X078,1450,116,168200.0
3,D31078,2019-04-08 00:00:00,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,102,187068.0
4,D32470,2019-04-11 00:00:00,S18696,北区,北京市,北京市,X010,887,58,51446.0


In [70]:
fact_order.info() # This dataset contains 314552 entries and 10 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31452 entries, 0 to 31451
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   订单编码    31452 non-null  object 
 1   订单日期    31452 non-null  object 
 2   客户编码    31452 non-null  object 
 3   所在区域    31450 non-null  object 
 4   所在省份    31450 non-null  object 
 5   所在地市    31452 non-null  object 
 6   商品编号    31451 non-null  object 
 7   订购数量    31450 non-null  object 
 8   订购单价    31448 non-null  object 
 9   金额      31448 non-null  float64
dtypes: float64(1), object(9)
memory usage: 2.4+ MB


In [71]:
# check duplicate entries
fact_order[fact_order.duplicated()].count()

订单编码    6
订单日期    6
客户编码    6
所在区域    6
所在省份    6
所在地市    6
商品编号    6
订购数量    6
订购单价    6
金额      6
dtype: int64

In [72]:
fact_order.drop_duplicates(inplace=True)   # delete duplicates
fact_order.reset_index(drop=True, inplace=True)  # rebuild index

In [73]:
# check null value
fact_order.isnull().sum()

订单编码    0
订单日期    0
客户编码    0
所在区域    2
所在省份    2
所在地市    0
商品编号    1
订购数量    2
订购单价    4
金额      4
dtype: int64

In [74]:
fact_order.fillna(method='bfill', inplace=True) # fill missing value before fill
fact_order.fillna(method='ffill', inplace=True) # fill missing value forward fill

In [64]:
# for dim product sheet
# check if there is duplicate data
dim_product[dim_product.duplicated()]

商品名称    0
商品小类    0
商品大类    0
销售单价    0
dtype: int64

In [67]:
# check null value
dim_product.isnull().sum()

0

In [76]:
fact_order['订单日期'] = fact_order['订单日期'].apply(lambda x: pd.to_datetime(x, format='%Y#%m#%d') if isinstance(x, str) else x)
fact_order[fact_order['订单日期'] > '2021-01-01'] # a wrong data

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额
20797,D26533,2050-06-09,S21396,北区,河北省,石家庄市,X022,759,158,119922.0


In [77]:
fact_order = fact_order[fact_order['订单日期'] < '2021-01-01'] # filter
fact_order['订单日期'].max(), fact_order['订单日期'].min()  # time range between 2019-01-01 and 2019-09-30 

(Timestamp('2019-09-30 00:00:00'), Timestamp('2019-01-01 00:00:00'))

In [78]:
fact_order['订购数量'] = fact_order['订购数量'].apply(lambda x: x.strip('个') if isinstance(x, str) else x).astype('int')
fact_order['订购单价'] = fact_order['订购单价'].apply(lambda x: x.strip('元') if isinstance(x, str) else x).astype('float')
fact_order['金额'] = fact_order['金额'].astype('float')

In [79]:
fact_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31445 entries, 0 to 31445
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   订单编码    31445 non-null  object        
 1   订单日期    31445 non-null  datetime64[ns]
 2   客户编码    31445 non-null  object        
 3   所在区域    31445 non-null  object        
 4   所在省份    31445 non-null  object        
 5   所在地市    31445 non-null  object        
 6   商品编号    31445 non-null  object        
 7   订购数量    31445 non-null  int64         
 8   订购单价    31445 non-null  float64       
 9   金额      31445 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 2.6+ MB


In [80]:
fact_order['所在省份'] = fact_order['所在省份'].str.replace('自治区|维吾尔|回族|壮族|省|市', '')  # clean province data
fact_order['所在省份'].unique()

<ipython-input-80-d54d2d67c0e3>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  fact_order['所在省份'] = fact_order['所在省份'].str.replace('自治区|维吾尔|回族|壮族|省|市', '')  # clean province data


array(['浙江', '安徽', '山西', '吉林', '北京', '云南', '广东', '广西', '内蒙古', '新疆', '湖北',
       '江苏', '甘肃', '四川', '河南', '福建', '陕西', '辽宁', '山东', '江西', '重庆', '河北',
       '湖南', '上海', '贵州', '天津', '海南', '宁夏', '黑龙江'], dtype=object)

In [81]:
fact_order['客户编码'] = fact_order['客户编码'].str.replace('编号', '')

## 3.2 Data Visulization

### 3.2.1 Number of Monthly Orders

In [83]:
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker

fact_order['订单月份'] = fact_order['订单日期'].apply(lambda x: x.month) 
item = fact_order.groupby('订单月份').agg({'订购数量': 'sum', '金额': 'sum'}).to_dict()
x = [f'{key} 月' for key in item['订购数量'].keys()]
y1 = [round(val/1000, 2) for val in item['订购数量'].values()]
y2 = [round(val/10000/100, 2) for val in item['金额'].values()]
c = (
    Bar()
    .add_xaxis(x)
    .add_yaxis("NUmber of orders（K）", y1, is_selected=False)
    .add_yaxis("Million RMB）", y2)
    .set_global_opts(title_opts=opts.TitleOpts(title="Number of Monthly Orders"))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=True),
    )
)
c.render_notebook()

### 3.2.2 Best sale province

In [87]:
item = fact_order.groupby('所在地市').agg({'订购数量': 'sum'}).sort_values(by='订购数量', ascending=False)[:20].sort_values(by='订购数量').to_dict()['订购数量']

c = (
    Bar()
    .add_xaxis([*item.keys()])
    .add_yaxis("Number of orders", [round(v/10000, 2) for v in item.values()], label_opts=opts.LabelOpts(position="right", formatter='{@[1]/} 万'))
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts("Number of Orders TOP20")
    )
)
c.render_notebook()